### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk

In [1]:
import json
import nltk
import numpy as np

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "dictionary.txt"
txt = [];
with open(filename) as f:
    lines = f.readlines()
    
for line in lines:
    line = line.strip()
    line = line.replace("\n","")
    line = line.replace(" ","")
    line = line.replace("、","")
    line = line.replace("，","")
    line = line.replace("。","")
    line = line.replace(".","")
    line = line.replace("/","")
    line = line.replace(")","")
    line = line.replace("(","")
    
    if len(line)>0:
        #print(list(line))
        txt.extend(list(line))

print(len(txt))

91616


In [5]:
def checkDic(w):
    if w in txt:
        return True
    else:
        return False

In [6]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    #print(summary_str)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
    ne_index_list = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list.append((ne_idx,len(ne)))
    
    '''
    if len(main_list) != len(ne_index_list):
        c += 1
        print(case_content)
        print(main_list)
        for i,l in ne_index_list:
            print(case_content[i])
        print("------------")
    '''
    
    texts = []
    for i,w in enumerate(case_content):
        label = getLabelWithWordIndex(i,ne_index_list)
        texts.append((w,label))
        
    docs.append(texts)

print(t)
print(c)

13452
0


In [ ]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 2701
print(ave)

In [ ]:
print(len(docs))

In [8]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

print(data[0])

[('李', 'JJ', 'O'), ('某', 'NNP', 'O'), ('，', 'NNP', 'O'), ('女', 'NNP', 'O'), ('，', 'VBD', 'O'), ('6', 'CD', 'O'), ('7', 'CD', 'O'), ('岁', 'NN', 'O'), ('。', 'NNP', 'O'), ('每', 'NNP', 'O'), ('于', 'NNP', 'O'), ('夏', 'NNP', 'O'), ('日', 'NNP', 'O'), ('洗', 'NNP', 'O'), ('澡', 'NNP', 'O'), ('后', 'NNP', 'O'), ('即', 'NNP', 'O'), ('寒', 'NNP', 'B-S'), ('热', 'NNP', 'I-S'), ('不', 'NNP', 'I-S'), ('适', 'NNP', 'E-S'), ('，', 'NNP', 'O'), ('自', 'NNP', 'O'), ('感', 'NNP', 'O'), ('发', 'NNP', 'O'), ('热', 'NNP', 'O'), ('而', 'NNP', 'O'), ('又', 'NNP', 'O'), ('需', 'NNP', 'O'), ('披', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('裹', 'NNP', 'O'), ('护', 'NNP', 'O'), ('。', 'NNP', 'O'), ('并', 'NNP', 'O'), ('于', 'NNP', 'O'), ('不', 'NNP', 'O'), ('知', 'NNP', 'O'), ('不', 'NNP', 'O'), ('觉', 'NNP', 'O'), ('中', 'NNP', 'O'), ('汗', 'NNP', 'B-S'), ('透', 'NNP', 'I-S'), ('衣', 'NNP', 'I-S'), ('服', 'NNP', 'E-S'), ('，', 'NNP', 'O'), ('急', 'NNP', 'O'), ('又', 'NNP', 'O'), ('换', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('，', 'NNP', 'O'), ('如', 'NNP', 'O'),

In [11]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    medterm = checkDic(word)

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'medterm=%s' % medterm 
    ]
    
    #if word.isupper():
        #print('YES')

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [12]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES


YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES


YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES


YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES
YES


In [22]:
for d2 in X_train[0]:
    print(d2)

['bias', 'word.lower=初', 'word[-3:]=初', 'word[-2:]=初', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=JJ', 'medterm=True', 'BOS', '+1:word.lower=诊', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNP']
['bias', 'word.lower=诊', 'word[-3:]=诊', 'word[-2:]=诊', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=NNP', 'medterm=True', '-1:word.lower=初', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=JJ', '+1:word.lower=：', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNP']
['bias', 'word.lower=：', 'word[-3:]=：', 'word[-2:]=：', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=NNP', 'medterm=False', '-1:word.lower=诊', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=NNP', '+1:word.lower=2', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=True', '+1:

In [13]:
print(len(X_train[0][4]))
print(X_train[0][5])

19
['bias', 'word.lower=0', 'word[-3:]=0', 'word[-2:]=0', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=True', 'postag=CD', 'medterm=True', '-1:word.lower=0', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=True', '-1:postag=CD', '+1:word.lower=6', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=True', '+1:postag=CD']


In [14]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 29851
Seconds required: 3.158

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 207643.212938
Feature norm: 1.000000
Error norm: 86092.704589
Active features: 29708
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.120

***** Iteration #2 *****
Loss: 205862.443488
Feature norm: 1.014122
Error norm: 75060.040294
Active features: 28564
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.570

***** Iteration #3 *****
Loss: 197174.510646
Feature norm: 1.108199
Error norm: 57475.850451
Active features: 21637
Line search trials: 1
Line search step: 1.000000
Seconds required 

***** Iteration #39 *****
Loss: 41722.293822
Feature norm: 23.154634
Error norm: 19207.423275
Active features: 23949
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.573

***** Iteration #40 *****
Loss: 41454.935380
Feature norm: 23.433364
Error norm: 41374.232670
Active features: 23887
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.570

***** Iteration #41 *****
Loss: 40591.140646
Feature norm: 23.802563
Error norm: 31578.232145
Active features: 23722
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.577

***** Iteration #42 *****
Loss: 40073.734582
Feature norm: 24.219776
Error norm: 33645.291779
Active features: 23588
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.565

***** Iteration #43 *****
Loss: 39416.250069
Feature norm: 24.733850
Error norm: 30621.590503
Active features: 23520
Line search trials: 1
Line search step: 1.0

***** Iteration #80 *****
Loss: 25834.930843
Feature norm: 65.165324
Error norm: 4410.719411
Active features: 20152
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.569

***** Iteration #81 *****
Loss: 25722.363713
Feature norm: 65.805287
Error norm: 4047.033375
Active features: 20129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.570

***** Iteration #82 *****
Loss: 25622.579710
Feature norm: 67.017127
Error norm: 3844.239753
Active features: 20114
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.565

***** Iteration #83 *****
Loss: 25537.530318
Feature norm: 67.759981
Error norm: 2196.157406
Active features: 20050
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.566

***** Iteration #84 *****
Loss: 25448.518482
Feature norm: 69.346344
Error norm: 6251.561889
Active features: 19985
Line search trials: 1
Line search step: 1.000000

***** Iteration #121 *****
Loss: 24371.765924
Feature norm: 90.836192
Error norm: 2696.740379
Active features: 19111
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.569

***** Iteration #122 *****
Loss: 24363.688150
Feature norm: 91.123976
Error norm: 2586.886626
Active features: 19078
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.570

***** Iteration #123 *****
Loss: 24356.120048
Feature norm: 91.370115
Error norm: 2751.527403
Active features: 19057
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.567

***** Iteration #124 *****
Loss: 24352.201131
Feature norm: 91.813703
Error norm: 5189.958506
Active features: 19024
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.568

***** Iteration #125 *****
Loss: 24344.060218
Feature norm: 92.039650
Error norm: 3210.439405
Active features: 19014
Line search trials: 1
Line search step: 1.0

***** Iteration #168 *****
Loss: 24173.033316
Feature norm: 100.236322
Error norm: 3841.409873
Active features: 18712
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.568

***** Iteration #169 *****
Loss: 24168.258375
Feature norm: 100.359023
Error norm: 2183.660263
Active features: 18710
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.577

***** Iteration #170 *****
Loss: 24166.790608
Feature norm: 100.556590
Error norm: 3823.112453
Active features: 18714
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.566

***** Iteration #171 *****
Loss: 24162.020615
Feature norm: 100.669905
Error norm: 2047.069904
Active features: 18715
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.567

***** Iteration #172 *****
Loss: 24160.710950
Feature norm: 100.846668
Error norm: 3594.991022
Active features: 18698
Line search trials: 1
Line search step

In [15]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

李 (O)
某 (O)
， (O)
男 (O)
， (O)
4 (O)
2 (O)
岁 (O)
。 (O)
医 (O)
案 (O)
编 (O)
号 (O)
： (O)
0 (O)
1 (O)
4 (O)
h (O)
0 (O)
2 (O)
0 (O)
。 (O)
初 (O)
诊 (O)
： (O)
1 (O)
9 (O)
8 (O)
9 (O)
年 (O)
1 (O)
0 (O)
月 (O)
1 (O)
1 (O)
日 (O)
。 (O)
发 (O)
热 (O)
微 (O)
汗 (O)
， (O)
头 (O)
重 (O)
身 (O)
拘 (O)
， (O)
脘 (O)
痞 (O)
， (O)
便 (O)
干 (O)
9 (O)
天 (O)
。 (O)
自 (O)
诉 (O)
反 (O)
复 (O)
发 (O)
热 (O)
9 (O)
天 (O)
， (O)
最 (O)
高 (O)
体 (O)
温 (O)
达 (O)
3 (O)
8 (O)
. (O)
9 (O)
° (O)
c (O)
， (O)
曾 (O)
化 (O)
验 (O)
血 (O)
常 (O)
规 (O)
： (O)
白 (O)
细 (O)
胞 (O)
4 (O)
. (O)
7 (O)
x (O)
1 (O)
0 (O)
9 (O)
/ (O)
l (O)
# (O)
中 (O)
性 (O)
粒 (O)
细 (O)
胞 (O)
6 (O)
1 (O)
% (O)
， (O)
淋 (O)
巴 (O)
细 (O)
胞 (O)
3 (O)
9 (O)
% (O)
# (O)
血 (O)
红 (O)
蛋 (O)
白 (O)
1 (O)
2 (O)
5 (O)
g (O)
/ (O)
l (O)
。 (O)
肥 (O)
达 (O)
反 (O)
应 (O)
： (O)
0 (O)
凝 (O)
集 (O)
素 (O)
1 (O)
: (O)
1 (O)
6 (O)
0 (O)
， (O)
h (O)
凝 (O)
集 (O)
素 (O)
1 (O)
: (O)
1 (O)
6 (O)
0 (O)
。 (O)
西 (O)
医 (O)
诊 (O)
断 (O)
为 (O)
伤 (O)
寒 (O)
， (O)
口 (O)
服 (O)
复 (O)
方 (O)
新 (O)
诺 (O)
明 (O)
， (O)
静 (O)
脉 (O

In [16]:
from sklearn.metrics import accuracy_score

# Convert the sequences of tags into a 1-dimensional array
#predictions = np.array([labels[tag] for row in y_pred for tag in row])
#truths = np.array([labels[tag] for row in y_test for tag in row])

print(y_pred)

#print(accuracy_score(y_test, y_pred))

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-S', 'I-S', 'I-S', 'I-S', 'E-S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-S', 'I-S', 'I-S', 'I-S', 'E-S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [17]:
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.98      0.99      0.99    316600
      B-SYM       0.44      0.18      0.26      2398
      I-SYM       0.40      0.20      0.26      3822
      E-SYM       0.46      0.19      0.27      2389

avg / total       0.96      0.97      0.97    325209



In [ ]:
s1 = "天下"
s2 = "地上"
w = []
w.extend(list(s1))
w.extend(list(s2))
print(w)